In [7]:
import time
import pandas as pd
from census import Census
import altair as alt
import numpy as np

# Census API access
api_key = "639f2aedf7c17b164527591258cda00b25249b4b"
c = Census(key=api_key)

In [8]:
# Tenure: Table B25003
tenure_variables = {
    'B25003_001E': 'total',
    'B25003_001M': 'total_moe',
    'B25003_002E': 'owner',
    'B25003_002M': 'owner_moe',
    'B25003_003E': 'renter',
    'B25003_003M': 'renter_moe'
}
tenure_columns_out = [
    'total', 'total_moe',
    'pct_owner', 'pct_owner_moe',
    'pct_renter', 'pct_renter_moe'
]

In [9]:
def combine_tenure(in_df):
    '''
    Outputs: simplified breakdown with MOEs
    '''
    df = in_df.copy()
    
    for group in ['owner', 'renter']:
        # Calculate the proportion for this group
        df[f'pct_{group}'] = df[group] / df['total']
    
        # Calculate the MOE for this proportion
        df[f'pct_{group}_moe'] = (df[f'{group}_moe']**2 - df[f'pct_{group}']**2 * df['total_moe']**2)**0.5 / df['total']

        #NaN-out any too-low absolute n
        df.loc[df['total'] < 25, f'pct_{group}'] = float('NaN')
        df.loc[df['total'] < 25, f'pct_{group}_moe'] = float('NaN')
        
        #NaN-out any too-low moe
        df[f'pct_{group}_moe_ratio'] = df[f'pct_{group}_moe']/df[f'pct_{group}']
        df.loc[df[f'pct_{group}_moe_ratio'] > .4, f'pct_{group}'] = float('NaN')
        df.loc[df[f'pct_{group}_moe_ratio'] > .4, f'pct_{group}_moe'] = float('NaN')
    
    return df

In [10]:
def combine_tracts(in_df):
    '''

    Outputs:
    A modified version of in_df with census tracts combined
    '''
    
    # Start by making a copy of in_df, so we don't destroy the original data.
    # df is also a nice short name we can use throughout this function
    df_copy = in_df.copy()
    df = pd.DataFrame(columns=tenure_variables.values())

    ### CLEAN UNUSUAL MOES
    df_copy = df_copy.replace(-555555555.0, 0)

    for c in tenure_variables.values():
        if "moe" not in c:
            # sum the totals
            df.at[0, c] = df_copy[c].sum()
        else:
            # sum of squares the moes
            df.at[0, c] = np.sqrt((df_copy[c]**2).sum())

    df_out = combine_tenure(df)
    
    return df_out

In [11]:
def get_tenure_precombo(year_in, place_num):
    # for getting mode for census tracts before they are combined
    df = pd.DataFrame(
        c.acs5.get(
            list(tenure_variables.keys()),
            {'for': place_num, 'in': 'state:06 county:013'},
            year=year_in
        )
    )
    df = df.rename(columns=tenure_variables)
    df = df.drop(columns=["state", "county", "tract"])
    
    return df

In [12]:
def get_city_df(year_in):
    # City
    df = pd.DataFrame(
        c.acs5.get(
            list(tenure_variables.keys()),
            {'for': 'place:60620', 'in': 'state:06'},
            year=year_in
        )
    )
    df = df.rename(columns=tenure_variables)
    df_out = combine_tenure(df)

    return df_out

In [13]:
# for 5yr ACS 2014 and 2019
# Get ACS Table B25003 for select tracts around the BART station
# county:013
# city: 60620
c_tracts = 'tract:375000, 376000, 377000, 374000, 381000'

df_tracts_2014 = get_tenure_precombo(2014, c_tracts)
df_tracts_2019 = get_tenure_precombo(2019, c_tracts)

In [14]:
df_comb_2014 = combine_tracts(df_tracts_2014)
df_tracts_out_2014 = df_comb_2014[tenure_columns_out]
df_tracts_out_2014.insert(0, "year", 2014)
df_comb_2019 = combine_tracts(df_tracts_2019)
df_tracts_out_2019 = df_comb_2019[tenure_columns_out]
df_tracts_out_2019.insert(0, "year", 2019)

In [31]:
# for 5yr ACS 2017 and 2022
df_tracts_2017 = get_tenure_precombo(2017, c_tracts)
df_tracts_2022 = get_tenure_precombo(2022, c_tracts)

In [32]:
df_comb_2017 = combine_tracts(df_tracts_2017)
df_tracts_out_2017 = df_comb_2017[tenure_columns_out]
df_tracts_out_2017.insert(0, "year", 2017)
df_comb_2022 = combine_tracts(df_tracts_2022)
df_tracts_out_2022 = df_comb_2022[tenure_columns_out]
df_tracts_out_2022.insert(0, "year", 2022)

In [15]:
df_tracts_out_2014

,year,total,total_moe,pct_owner,pct_owner_moe,pct_renter,pct_renter_moe
0,2014,8866.0,258.994208,0.384615,0.031654,0.615385,0.036305


In [16]:
# for 5yr ACS 2014 and 2019
# Get ACS Table B08141 in Richmond City
df_city_2014 = get_city_df(2014)
df_city_2019 = get_city_df(2019)

In [17]:
df_city_out_2014 = df_city_2014[tenure_columns_out]
df_city_out_2014.insert(0, "year", 2014)
df_city_out_2019 = df_city_2019[tenure_columns_out]
df_city_out_2019.insert(0, "year", 2019)

In [21]:
# for 5yr ACS 2017 and 2022
# Get ACS Table B08141 in Richmond City
df_city_2017 = get_city_df(2017)
df_city_2022 = get_city_df(2022)

In [22]:
df_city_out_2017 = df_city_2017[tenure_columns_out]
df_city_out_2017.insert(0, "year", 2017)
df_city_out_2022 = df_city_2022[tenure_columns_out]
df_city_out_2022.insert(0, "year", 2022)

In [18]:
df_city_out_2014

,year,total,total_moe,pct_owner,pct_owner_moe,pct_renter,pct_renter_moe
0,2014,36413.0,741.0,0.494301,0.016896,0.505699,0.015583


In [19]:
#export settings, for a combined csv
df_tracts_out = pd.concat([df_tracts_out_2014, df_tracts_out_2019])
df_city_out = pd.concat([df_city_out_2014, df_city_out_2019])

In [20]:
df_tracts_out

,year,total,total_moe,pct_owner,pct_owner_moe,pct_renter,pct_renter_moe
0,2014,8866.0,258.994208,0.384615,0.031654,0.615385,0.036305
0,2019,8912.0,233.63861,0.384874,0.031682,0.615126,0.033449


In [21]:
df_city_out

,year,total,total_moe,pct_owner,pct_owner_moe,pct_renter,pct_renter_moe
0,2014,36413.0,741.0,0.494301,0.016896,0.505699,0.015583
0,2019,37088.0,737.0,0.498328,0.020219,0.501672,0.017656
